In [1]:
from os import chdir
chdir('../')
%matplotlib inline
from lib import *

pd.options.display.precision = 4
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Lato'
mpl.rcParams['font.weight'] = 700
sns.set(font='Lato', font_scale=1)
sns.set()
np.random.seed(42)
suppress_warnings()

In [2]:
feature = pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/feature')
target= pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/target')

In [3]:
display(feature.shape)
display(target.shape)

(10768, 1000)

(10768, 1)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, LogisticRegression,LinearRegression
from sklearn.pipeline import Pipeline

# Step 2 Identify Features

In [5]:
from sklearn.feature_selection import SelectKBest, \
                                      SelectFromModel, \
                                      RFE, SelectPercentile, \
                                      SelectFdr, SelectFpr
from sklearn.feature_selection import f_regression, f_classif
from sklearn.neighbors import KNeighborsClassifier

### Pipeline 2: SKB, PCA, SFM, LR

In [6]:
pipe_for_gs = Pipeline([
    ('skb', SelectKBest(score_func=f_classif)),
    ('pca',PCA(random_state=42)),
    ('sfm', SelectFromModel(LogisticRegression())),
    ('lr', LogisticRegression(C=1E10,n_jobs=-1,random_state=42))
])

In [7]:
params = {    
    'skb__k':[250,500],
    'pca__n_components':[50,100,150],
    'sfm__estimator':[LogisticRegression(C=0.1,penalty='l1',random_state=42),
                      LogisticRegression(C=1,penalty='l1',random_state=42),
                      LogisticRegression(C=10,penalty='l1',random_state=42),
                      LogisticRegression(C=1e2,penalty='l1',random_state=42),
                      LogisticRegression(C=1e3,penalty='l1',random_state=42),
                      LogisticRegression(C=1e4,penalty='l1',random_state=42)]
}

In [8]:
gspipe = GridSearchCV(pipe_for_gs,
                      param_grid=params,
                      n_jobs=-1,
                      cv=ShuffleSplit(random_state=42))

In [9]:
gspipe.fit(feature[:220], np.ravel(target[:220].values))

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=42, test_size=0.1, train_size=None),
       error_score='raise',
       estimator=Pipeline(steps=[('skb', SelectKBest(k=10, score_func=<function f_classif at 0x7f8bd315e6a8>)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('sfm', SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None, dual=Fa...alty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'pca__n_components': [50, 100, 150], 'sfm__estimator': [LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
        ...e=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)], 'skb__k': [250, 

In [10]:
gspipe.best_score_

0.50454545454545452

In [11]:
gspipe.best_estimator_.named_steps

{'lr': LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=42,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 'pca': PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
   svd_solver='auto', tol=0.0, whiten=False),
 'sfm': SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
         prefit=False, threshold=None),
 'skb': SelectKBest(k=500, score_func=<function f_classif at 0x7f8bd315e6a8>)}

In [12]:
gspipe_results = pd.DataFrame(gspipe.cv_results_)
gspipe_results.sort_values('rank_test_score', ascending=True).T.head(6)

,3,5,7,9,11,1,19,6,8,10,...,33,22,20,14,30,26,28,16,34,32
mean_fit_time,0.04057,0.0427,0.04513,0.04507,0.04456,0.04013,0.05403,0.1275,0.1809,0.2291,...,0.05126,0.03777,0.03572,0.03398,0.03897,0.03504,0.03645,0.03829,0.03839,0.0375
mean_score_time,0.0007293,0.0007688,0.0008354,0.0008002,0.0007753,0.0007293,0.0009366,0.0007265,0.0006675,0.0007328,...,0.00107,0.00075,0.0007665,0.0007664,0.0008002,0.000948,0.0008072,0.0008549,0.0008255,0.0008244
mean_test_score,0.5045,0.5045,0.5045,0.5045,0.5045,0.5,0.4682,0.4591,0.4591,0.4591,...,0.4182,0.4136,0.4045,0.4045,0.3955,0.3864,0.3864,0.3864,0.3818,0.3773
mean_train_score,0.8303,0.8303,0.8303,0.8303,0.8303,0.8308,1,0.9838,0.9838,0.9838,...,1,1,1,1,1,1,1,1,1,1
param_pca__n_components,50,50,50,50,50,50,100,50,50,50,...,150,100,100,100,150,150,150,100,150,150
param_sfm__estimator,"LogisticRegression(C=1, class_weight=None, dua...","LogisticRegression(C=10, class_weight=None, du...","LogisticRegression(C=100.0, class_weight=None,...","LogisticRegression(C=1000.0, class_weight=None...","LogisticRegression(C=10000.0, class_weight=Non...","LogisticRegression(C=0.1, class_weight=None, d...","LogisticRegression(C=100.0, class_weight=None,...","LogisticRegression(C=100.0, class_weight=None,...","LogisticRegression(C=1000.0, class_weight=None...","LogisticRegression(C=10000.0, class_weight=Non...",...,"LogisticRegression(C=1000.0, class_weight=None...","LogisticRegression(C=10000.0, class_weight=Non...","LogisticRegression(C=1000.0, class_weight=None...","LogisticRegression(C=1, class_weight=None, dua...","LogisticRegression(C=100.0, class_weight=None,...","LogisticRegression(C=1, class_weight=None, dua...","LogisticRegression(C=10, class_weight=None, du...","LogisticRegression(C=10, class_weight=None, du...","LogisticRegression(C=10000.0, class_weight=Non...","LogisticRegression(C=1000.0, class_weight=None..."


In [13]:
gspipe_results['mean_fit_time'].sum()

2.0559993743896481

In [14]:
# gspipe_results.info(memory_usage='deep')

In [15]:
print("Fit time .1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .1% data: ",gspipe.best_score_)
### Pipeline2 for .1% (220 rows)
picked_pipeline_220row = \
        pd.DataFrame.to_pickle(gspipe_results,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline2_lr_pt1_percent')

Fit time .1% data:  2.05599937439
Best Score .1% data:  0.504545454545


In [16]:
### Pipeline2 for .5% (1100 rows)

gspipe.fit(feature[:1100], np.ravel(target[:1100].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time .5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .5% data: ",gspipe.best_score_)
gspipe_results_1100 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline2_1100row = \
        pd.DataFrame.to_pickle(gspipe_results_1100,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline2_lr_pt5_percent')

Fit time .5% data:  6.31589820385
Best Score .5% data:  0.501818181818


In [17]:
### Pipeline2 for 1% (2200 rows)

gspipe.fit(feature[:2200], np.ravel(target[:2200].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 1% data: ",gspipe.best_score_)
gspipe_results_2200 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline2_2200row = \
        pd.DataFrame.to_pickle(gspipe_results_2200,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline2_lr_1_percent')

Fit time 1% data:  11.910923171
Best Score 1% data:  0.522727272727


In [18]:
### Bench Marking for 5% (11000 rows)

gspipe.fit(feature, np.ravel(target.values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 5% data:",gspipe.best_score_)
gspipe_results_11000 = pd.DataFrame(gspipe.cv_results_)
picked_GS_BenchMarkig_11000row = \
        pd.DataFrame.to_pickle(gspipe_results_11000,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline2_lr_5_percent')

Fit time 5% data:  64.2801791191
Best Score 5% data: 0.524233983287
